In [ ]:
!ls /usr/local/

bin    cuda	cuda-12.2  games	       include	lib64	   man	 share
colab  cuda-12	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [ ]:
!which nvcc

/usr/local/cuda/bin/nvcc


In [120]:
!nvidia-smi

Fri Jul 26 02:37:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [124]:
%%writefile mergesort.cu

#include <stdio.h>

__global__ void merge(int *in, int *out, int n, int stripe)
{
  int i = blockIdx.x*blockDim.x + threadIdx.x;
  int num_elements = pow(2.0, (double)stripe);
  int left_begin = i*num_elements*2;
  int left_end = left_begin + num_elements - 1;
  int right_begin = left_end+1;
  int right_end = right_begin + num_elements - 1;

  //printf("block = %d, thread = %d, i = %d, stripe = %d, num_elements = %d, left_begin = %d, left_end = %d, right_begin = %d, right_end = %d\n", blockIdx.x, threadIdx.x, i, stripe, num_elements, left_begin, left_end, right_begin, right_end);

  int l_index = left_begin;
  int r_index = right_begin;
  int index = left_begin;

  if(left_end < n && right_end < n)
  {
    while(l_index <= left_end && r_index <= right_end)
    {
      int l_val = in[l_index];
      int r_val = in[r_index];
      if(l_val <= r_val)
      {
        out[index] = l_val;
        l_index++;
      }
      else if(r_val < l_val)
      {
        out[index] = r_val;
        r_index++;
      }
      index++;
    }

    while(l_index <= left_end)
    {
      out[index] = in[l_index];
      l_index++;
      index++;
    }

    while(r_index <= right_end)
    {
      out[index] = in[r_index];
      r_index++;
      index++;
    }
  }
}

__global__ void sort(int n, int *x)
{
  int i = blockIdx.x*blockDim.x + threadIdx.x;
  //printf("block = %d, blockDim = %d, thread = %d\n", blockIdx.x, blockDim.x, threadIdx.x);
  //printf("i = %d\n", i);
  if((i*2)+1 >= n)
    return;

  int num1 = x[i*2];
  int num2 = x[(i*2)+1];
  if(num1 > num2)
  {
    x[(i*2)+1] = num1;
    x[i*2] = num2;
    //printf("index = %d, val = %d\n", (i*2), x[i*2]);
    //printf("index = %d, val = %d\n", ((i*2)+1), x[(i*2)+1]);
  }
}

__global__ void mergesort(int n, int *x, int *y)
{
  int size = n/2;
  int num_blocks = (size > 512) ? (size/512) : 1;
  int num_threads = (size > 512) ? 512 : size;

  sort<<<num_blocks, num_threads>>>(n, x);

  /*for(int i=0; i<n; i++)
    printf("(%d, %d), ", i, x[i]);
  printf("\n");
  printf("check -- %d\n", x[0]);*/

  int stripe = 0;
  while(size/2 >= 1)
  {
    stripe++;
    size = size/2;

    num_blocks = (size > 512) ? (size/512) : 1;
    num_threads = (size > 512) ? 512 : size;

    if(stripe%2 == 1)
      merge<<<num_blocks, num_threads>>>(x, y, n, stripe);
    else
      merge<<<num_blocks, num_threads>>>(y, x, n, stripe);
  }
}

int main(void)
{
  int N = 2048;
  int *x, *y, *d_x, *d_y;
  x = (int*)malloc(N*sizeof(int));
  y = (int*)malloc(N*sizeof(int));

  cudaMalloc(&d_x, N*sizeof(int));
  cudaMalloc(&d_y, N*sizeof(int));

  for (int i = 0; i < N; i++) {
    x[i] = N-i;
    y[i] = 0;
  }

  cudaMemcpy(d_x, x, N*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(int), cudaMemcpyHostToDevice);

  // Perform mergesort
  mergesort<<<1, 1>>>(N, d_x, d_y);

  int size = N;
  int count = 0;
  while(size > 1)
  {
    size = size/2;
    count++;
  }

  if(count % 2 == 1)
    cudaMemcpy(x, d_x, N*sizeof(int), cudaMemcpyDeviceToHost);
  else
    cudaMemcpy(x, d_y, N*sizeof(int), cudaMemcpyDeviceToHost);

  printf("output\n");
  for (int i = 0; i < N; i++)
    if(x[i] != i+1)
      printf("False!! i = %d\n", i);

  cudaFree(d_x);
  cudaFree(d_y);
  free(x);
  free(y);
}

Writing mergesort.cu


In [125]:
!nvcc -rdc=true mergesort.cu -o mergesort

In [126]:
!./mergesort

output
